# 0. 소개

Naver 영화리뷰 감정분석 데이터를 다운로드 하고 Sentencepiece를 활용해 Vocab 만들기에서 생성된 vocab을 활용해 이후 학습하기 좋은 형태로 미리 작업을 해 놓는 과정 입니다.

# 1. 다운로드

[Naver sentiment movie corpus](https://github.com/e9t/nsmc)에서 다운로드 하거나 아래 명령으로 다운로드 하세요.

* [학습데이터: ratings_train.txt](https://github.com/e9t/nsmc/blob/master/ratings_train.txt)
* [평가데이터: ratings_test.txt](https://github.com/e9t/nsmc/blob/master/ratings_test.txt)

In [1]:
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt

--2022-10-29 02:20:36--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘ratings_train.txt.1’

ratings_train.txt.1 100%[===================>]  13.95M  --.-KB/s    in 0.1s    

2022-10-29 02:20:36 (106 MB/s) - ‘ratings_train.txt.1’ saved [14628807/14628807]

--2022-10-29 02:20:36--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (4.7M) [text/plain]
Sa

# 2. Vocab

Sentencepiece를 활용해 Vocab 만들기를 통해 만들어 놓은 vocab을 로드 합니다.

In [2]:
# 드라이브 마운트 후 접근
%cd "drive/MyDrive/Colab Notebooks/3.5. 영화 리뷰 감성 분석/Vocab/"

/content/drive/MyDrive/Colab Notebooks/3.5. 영화 리뷰 감성 분석/Vocab


In [3]:
%ls

kowiki_20221028.csv  kowiki.model  kowiki.txt  kowiki.vocab


In [13]:
!pip install sentencepiece
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
import pandas as pd
import json

In [6]:
# vocab loading
import sentencepiece as spm

vocab_file = "kowiki.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

True

# 3. 데이터 전처리

다운로드된 데이터를 vocab으로 미리 토큰화해서 json형태로 저장 해 놓습니다.
토큰화를 미리하지 않고 학습 할 경우 처리시간이 매번 소요 되므로 이를 효과적으로 줄이기 위함 입니다.

In [7]:
# train data 준비
import sentencepiece as spm

def prepare_train(vocab, infile, outfile):
    df = pd.read_csv(infile, sep="\t", engine="python")
    with open(outfile, "w") as f:
        for index, row in df.iterrows():
            document = row["document"]
            if type(document) != str:
                continue
            instance = { "id": row["id"], "doc": vocab.encode_as_pieces(document), "label": row["label"] }
            f.write(json.dumps(instance))
            f.write("\n")

아래 코드를 실행하면 전처리된 파일이 생성 됩니다.

* 학습데이터: ratings_train.json
* 평가데이터: ratings_test.json

In [9]:
corpus = "kowiki.txt"
prefix = "kowiki"
vocab_size = 8000
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
    " --model_type=bpe" +
    " --max_sentence_length=999999" + # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" + # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]") # 사용자 정의 토큰

In [10]:
%cd "/content"

/content


In [11]:
%ls

drive/            ratings_test.txt.1  ratings_train.txt.1
ratings_test.txt  ratings_train.txt   sample_data/


In [19]:
prepare_train(vocab, "ratings_train.txt", "ratings_train.json")
prepare_train(vocab, "ratings_test.txt", "ratings_test.json")

# 4. 참고

자세한 내용은 다음을 참고 하세요.

* [preprocess_nsmc.ipynb](https://github.com/paul-hyun/transformer-evolution/blob/master/tutorial/preprocess_nsmc.ipynb)
* [common_data.py](https://github.com/paul-hyun/transformer-evolution/blob/master/common_data.py)
* [Naver sentiment movie corpus](https://github.com/e9t/nsmc)
* [데이터 파일](https://drive.google.com/drive/folders/15XGr-L-W6DSoR5TbniPMJASPsA0IDTiN)